In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
import Tr_centrality_measure
import Graph_Entropy_measure
import Selection_Algorithm
import Constraint_Coefficient

In [3]:
PI_table1 = pd.DataFrame(Graph_Entropy_measure.PI_Rank, columns = ('nodes_PI','PI_i'))
#PI_table1.PI_i[PI_table.PI_i  < 0] = 0
PI_table = PI_table1.copy()
i = math.sqrt(2)
PI_table.PI_i=PI_table.PI_i.where(lambda x: x > 0, lambda x: x + i)

In [4]:
TC_table = pd.DataFrame(Tr_centrality_measure.TC_Rank, columns = ('nodes_TC','TC_i'))
#TC_table.nodes_TC = TC_table.nodes_TC + 1
CC_table = pd.DataFrame(Constraint_Coefficient.sorted_x, columns = ('nodes_CC','CC_i'))
#CC_table.nodes_CC= CC_table.nodes_CC + 1
Result_table = pd.concat([TC_table, PI_table, CC_table], axis=1, sort=False)

First_Five= pd.DataFrame(Result_table.loc[0:2])
First_Five.index = First_Five.index + 1
First_Five

,nodes_TC,TC_i,nodes_PI,PI_i,nodes_CC,CC_i
1,104,0.5076,104,0.935014,22,0.026222
2,15,0.5106,340,1.403414,332,0.026895
3,195,0.5132,232,1.410114,104,0.027567


In [5]:
#return the total power of node i.
def TPN(i):
    PI_ofi = Graph_Entropy_measure.PI_of(i)
    TC_ofi = Tr_centrality_measure.TC_of(i)
    CC_ofi = Constraint_Coefficient.CC_of(i)
    #Weight to normalize the measures
    n1=TC_ofi /Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    n2=PI_ofi/Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    n3=CC_ofi/Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    #equation []
    #TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)    
    TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)    
    return TPN_i

In [6]:
#loop total power of node all i
def loop_TPN():
    i=0
    nodes = list(pd.concat([First_Five['nodes_TC'], First_Five['nodes_PI'], First_Five['nodes_CC']]).unique())
    list_TPN=[]    
    for i in nodes:          
        TPN_ = TPN(i)
        i_TPN = [i, TPN_]
        list_TPN.append(i_TPN)  
            
    return  list_TPN

In [7]:
#Ranking the total power of node i
def TPN_Ranking( val ):
      return val [1]

TPN_Rank = loop_TPN()
TPN_Rank.sort(key=lambda elem: elem[1])

In [8]:
#return table containing the TPN values[TPN_table and Top_Five]
TPN_table = pd.DataFrame(TPN_Rank, columns = ('nodes_TPN','TPN_i'))
Top_Five = pd.DataFrame(TPN_table.loc[0:2])
Top_Five.index = Top_Five.index + 1
#TPN_table.nodes_TPN = TPN_table.nodes_TPN + 1
print(Top_Five)
#TPN_table.to_csv('result/TPN_[]_network.csv')
#Top_Five.to_csv('result/TPN_[]_network.csv')

   nodes_TPN     TPN_i
1        232  0.483611
2        340  0.498778
3         15  0.685929


In [9]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Top_Five = pd.DataFrame(Propose_Result_table.loc[0:2])
Top_Five.index = Top_Five.index + 1
#convert column nodes_TPN to int
Top_Five.nodes_TPN= Top_Five.nodes_TPN.astype(int)
print(Top_Five)
#Propose_Table.to_csv('result/Propose_Table_kc_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

   nodes_TC    TC_i  nodes_PI      PI_i  nodes_CC      CC_i  nodes_TPN  \
1       104  0.5076       104  0.935014        22  0.026222        232   
2        15  0.5106       340  1.403414       332  0.026895        340   
3       195  0.5132       232  1.410114       104  0.027567         15   

      TPN_i  
1  0.483611  
2  0.498778  
3  0.685929  


In [10]:
Propose_Result_table 

,nodes_TC,TC_i,nodes_PI,PI_i,nodes_CC,CC_i,nodes_TPN,TPN_i
0,104,0.5076,104,0.935014,22,0.026222,232.0,0.483611
1,15,0.5106,340,1.403414,332,0.026895,340.0,0.498778
2,195,0.5132,232,1.410114,104,0.027567,15.0,0.685929
3,41,0.5141,133,0.046400,134,0.028366,104.0,0.941808
4,2,0.5143,587,0.092600,232,0.028909,332.0,1.529465
5,298,0.5145,453,0.160300,577,0.030489,195.0,2.074936
6,48,0.5164,40,0.160400,40,0.030873,22.0,5.564983
7,203,0.5166,354,0.161800,41,0.031143,NaN,NaN
8,204,0.5166,181,0.163300,377,0.032173,NaN,NaN
9,45,0.5177,15,0.164300,51,0.032947,NaN,NaN
